# Thai Food Classification with Huggingface's transformers

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ai-builders/curriculum/blob/main/notebooks/04v_classification_transformers.ipynb)

ใน Notebook นี้เราจะโหลดชุดข้อมูลอาหารไทย 50 ชนิด สร้าง datasets และจะใช้วิธีการ fine-tune โมเดล Swin transformer tiny เพื่อแบ่งประเภทภาพอาหารไทย 50 ชนิด

อ่านเพิ่มเติม: [huggingface datasets](https://huggingface.co/docs/datasets/image_load)

In [1]:
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.5 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-esqwps6c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-esqwps6c
  Resolved https://github.com/huggingface/transformers to commit 5af2c6269672cda01c24ad48fab13f14a3ffb746
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.4 M

## Download datasets

- ดาวน์โหลด FoodyDudy dataset จาก github
- จากนั้นใช้ `load_dataset("imagefolder", ...)` เพื่ออ่านข้อมูลมาใน class `Dataset`
- ทำการโหลด feature extractor ของโมเดลที่จะใช้ fine-tune เพื่อใช้ในการปรับขนาดของภาพ

In [2]:
!git clone https://github.com/GemmyTheGeek/FoodyDudy.git

Cloning into 'FoodyDudy'...
remote: Enumerating objects: 14727, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14727 (delta 7), reused 18 (delta 7), pack-reused 14709
Receiving objects: 100% (14727/14727), 645.40 MiB | 14.84 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Updating files: 100% (14465/14465), done.


In [3]:
food_list = [
    'green_curry', 'tepo_curry', 'liang_curry', 'taohoo_moosup', 'mara_yadsai',
    'masaman', 'orange_curry', 'cashew_chicken', 'omelette', 'sunny_side_up',
    'palo_egg', 'sil_egg', 'nun_banana', 'kua_gai', 'cabbage_fish_sauce',
    'river_prawn', 'shrimp_ob_woonsen', 'kanom_krok', 'mango_sticky_rice', 'kao_kamoo',
    'kao_klook_kapi', 'kaosoi', 'kao_pad', 'kao_pad_shrimp', 'chicken_rice',
    'kao_mok_gai', 'tom_ka_gai', 'tom_yum_kung', 'tod_mun', 'poh_pia',
    'pak_boong_fai_daeng', 'padthai', 'pad_krapao', 'pad_si_ew', 'pad_fakthong',
    'eggplant_stirfry', 'pad_hoi_lai', 'foithong', 'panaeng', 'yum_tua_ploo',
    'yum_woonsen', 'larb_moo', 'pumpkin_custard', 'sakoo_sai_moo', 'somtam',
    'moopoing','satay', 'hor_mok'
]
id2food = {str(i).zfill(2): f for i, f in enumerate(food_list)}

In [4]:
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("imagefolder", data_dir="FoodyDudy/images")
accuracy = load_metric("accuracy")

Resolving data files:   0%|          | 0/11520 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1440 [00:00<?, ?it/s]

In [ ]:
dataset

In [ ]:
from transformers import AutoFeatureExtractor

model_checkpoint = "microsoft/swin-tiny-patch4-window7-224"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feat_size = tuple(feature_extractor.size.values())
feature_extractor

จากนั้นสร้าง `preprocess_train` และ `preprocess_val` เพื่อ preprocess ข้อมูลภาพในแต่ละ batch ของเรา จะเห็นว่าภาพของเราอยู่ใน key ที่ชื่อว่า `image` และเมื่อเรา preprocess ภาพเรียบร้อยจะเก็บไว้ใน key ที่ชื่อว่า `pixel_values`

In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    RandomAffine,
    ColorJitter,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose([
    Resize(feat_size),
    RandomResizedCrop(feat_size, scale=(0.8, 1.2)),
    ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1),
    RandomHorizontalFlip(),
    ToTensor(),
    normalize,
])

val_transforms = Compose([
    Resize(feat_size),
    ToTensor(),
    normalize,
])


def preprocess_train(example_batch):
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [ ]:
dataset["train"].set_transform(preprocess_train)
dataset["validation"].set_transform(preprocess_val)

In [ ]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
id2label[2]

## Fine-tune our model

- โหลดโมเดลจาก huggingface hub
- สร้าง training arguments
- จากนั้นเทรนและเซฟโมเดล

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

In [ ]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 32

In [ ]:
args = TrainingArguments(
    f"{model_name}-finetuned-eurosat",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

In [ ]:
import torch
import numpy as np

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(f"trained/{model_name}")
# alternatively use trainer.push_to_hub instead

In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)

## Prediction: Using feature extractor and model

การใช้โมเดลที่เทรนเรียบร้อยแล้วมาทำนายภาพประกอบด้วย `AutoFeatureExtractor` และ `AutoModelForImageClassification` ทั้งนี้สามารถโหลดโมเดลจากโฟลเดอร์ที่เทรนเสร็จเรียบร้อยแล้ว หรือโหลดจาก huggingface hub ก็ได้ ในตัวอย่างนี้เราจะโหลดจากโฟล์เดอร์ที่เซฟโมเดลไป

จากนั้นสามารถอ่านภาพ `image` และแปลงให้เป็นฟีเจอร์ที่เหมาะสม ก่อนที่จะใส่เข้าไปในโมเดล โดย output ที่ได้จากโมเดลสามารถนำไปใช้ต่อได้เหมือนกับการเขียนโมเดล Pytorch ทั่วไปเลย

In [ ]:
import requests
import torch
from transformers import AutoModelForImageClassification, AutoFeatureExtractor
from PIL import Image

# read trained model from a folder (please double-check if you point to the correct path)
model_name = "./trained/swin-tiny-patch4-window7-224/"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)

In [ ]:
url = "https://github.com/GemmyTheGeek/FoodyDudy/raw/main/images/test/03/0289.jpg"
image = Image.open(requests.get(url, stream=True).raw)
encoding = feature_extractor(image.convert("RGB"), return_tensors="pt")
image

In [ ]:
with torch.no_grad():
    outputs = model(**encoding)
    logits = outputs.logits

In [ ]:
pred_idx = logits.argmax(-1).item()
print("Predicted class:", id2food[model.config.id2label[pred_idx]])

Predicted class: taohoo_moosup


## Prediction: Using pipeline API

นอกจากนั้น `transformers` ยังมี `pipeline` ที่เราเลือกชนิดของ pipeline แบบต่างๆ เช่น `image-classification` ทำให้การทำนายทำได้สะดวกยิ่งขึ้น

In [ ]:
from transformers import pipeline

pipe = pipeline("image-classification", model_name)

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
pipe(image)

[{'score': 0.9926163554191589, 'label': '03'},
 {'score': 0.006231394596397877, 'label': '04'},
 {'score': 0.00036008251481689513, 'label': '02'},
 {'score': 0.00020661753660533577, 'label': '10'},
 {'score': 0.0001315288827754557, 'label': '26'}]

In [ ]:
[{"score": l["score"], "label": id2food[l["label"]]} for l in pipe(image)]

[{'score': 0.9926163554191589, 'label': 'taohoo_moosup'},
 {'score': 0.006231394596397877, 'label': 'mara_yadsai'},
 {'score': 0.00036008251481689513, 'label': 'liang_curry'},
 {'score': 0.00020661753660533577, 'label': 'palo_egg'},
 {'score': 0.0001315288827754557, 'label': 'tom_ka_gai'}]

## Create gradio application for prediction

สุดท้ายแล้วเราสามารถนำโค้ดทั้งหมดมาจัดเรียง และ deploy ด้วย Gradio application ทั้งนี้เราเพียงต้องเขียน
- ฟังก์ชั่นเพื่อ inference โดยมี `id2food` เพื่อเปลี่ยน class ที่ทำนายเป็นชื่ออาหาร
- input ซึ่งเป็นชนิดภาพ `gr.inputs.Image()`
- output เป็น label ที่ทำนายได้ `gr.outputs.Label(num_top_classes=5)`
- ประกอบร่างกันเข้ามาด้วย `gr.Interface`

In [ ]:
import gradio as gr


food_list = [
    'green_curry', 'tepo_curry', 'liang_curry', 'taohoo_moosup', 'mara_yadsai',
    'masaman', 'orange_curry', 'cashew_chicken', 'omelette', 'sunny_side_up',
    'palo_egg', 'sil_egg', 'nun_banana', 'kua_gai', 'cabbage_fish_sauce',
    'river_prawn', 'shrimp_ob_woonsen', 'kanom_krok', 'mango_sticky_rice', 'kao_kamoo',
    'kao_klook_kapi', 'kaosoi', 'kao_pad', 'kao_pad_shrimp', 'chicken_rice',
    'kao_mok_gai', 'tom_ka_gai', 'tom_yum_kung', 'tod_mun', 'poh_pia',
    'pak_boong_fai_daeng', 'padthai', 'pad_krapao', 'pad_si_ew', 'pad_fakthong',
    'eggplant_stirfry', 'pad_hoi_lai', 'foithong', 'panaeng', 'yum_tua_ploo',
    'yum_woonsen', 'larb_moo', 'pumpkin_custard', 'sakoo_sai_moo', 'somtam',
    'moopoing','satay', 'hor_mok'
]
id2food = {str(i).zfill(2): f for i, f in enumerate(food_list)}


def inference(gr_input):
    """Inference function from gradio input."""
    image = Image.fromarray(gr_input.astype("uint8"), "RGB")
    predictions = pipe(image)
    predictions = {id2food[l["label"]]: l["score"] for l in predictions}
    return predictions

In [ ]:
inputs = gr.inputs.Image()
outputs = gr.outputs.Label(num_top_classes=5)

interface = gr.Interface(
    fn=inference, inputs=inputs, outputs=outputs, interpretation="default",
).launch(debug="True")

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
